install all lib 

In [36]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.26.1
    Uninstalling protobuf-5.26.1:
      Successfully uninstalled protobuf-5.26.1
Note: you may need to restart the kernel to use updated packages.


import all lib in this projects

In [24]:
from google.cloud import bigquery
import pandas as pd
import requests
import pandas as pd
import pyarrow.parquet as pq
from datetime import datetime

connect to google credentials, beacuse i use BigQuery. 
please use your json key

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="PATH_YOUR_JSON_KEY.json"

function for executing a request in BigQuery


In [5]:
client = bigquery.Client()

def create_table(query):
    query_job = client.query(query)
    return query_job.to_dataframe()

first sql task.

first qusery - create table and filling out the table

use your dataset in all query

In [10]:
transaction_table = f"""
CREATE TABLE USE_YOUR_DATASET.transactions (
  user_id INT64,
  spend FLOAT64,
  transaction_date TIMESTAMP
);

INSERT INTO USE_YOUR_DATASET.transactions (user_id, spend, transaction_date)
VALUES
  (111, 100.50, TIMESTAMP '2022-01-08 12:00:00'),
  (111, 55.00, TIMESTAMP '2022-01-10 12:00:00'),
  (121, 36.00, TIMESTAMP '2022-01-18 12:00:00'),
  (145, 24.99, TIMESTAMP '2022-01-26 12:00:00'),
  (111, 89.60, TIMESTAMP '2022-02-05 12:00:00');

SELECT * FROM `USE_YOUR_DATASET.transactions`
"""

df_transactions = create_table(transaction_table)
df_transactions

,user_id,spend,transaction_date
0,111,89.60,2022-02-05 12:00:00+00:00
1,111,55.00,2022-01-10 12:00:00+00:00
2,111,100.50,2022-01-08 12:00:00+00:00
3,121,36.00,2022-01-18 12:00:00+00:00
4,145,24.99,2022-01-26 12:00:00+00:00


i use this query to receive 3 purchases

In [11]:
third_transaction = f"""
WITH transactions AS (
  SELECT * FROM `USE_YOUR_DATASET.transactions`
)
, third_transactions AS (
  SELECT *
    , ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY transaction_date ASC) AS row_n
  FROM transactions
  GROUP BY 1, 2, 3
)
SELECT
  user_id
  , spend
  , transaction_date
FROM third_transactions
WHERE row_n = 3
"""

df_third_transaction = create_table(third_transaction)
df_third_transaction

,user_id,spend,transaction_date
0,111,89.6,2022-02-05 12:00:00+00:00


second sql task analog first

use your dataset please

In [12]:
phone_calls = f"""
CREATE TABLE USE_YOUR_DATASET.phone_calls (
  caller_id INT64,
  receiver_id INT64,
  call_time TIMESTAMP
);

INSERT INTO USE_YOUR_DATASET.phone_calls (caller_id, receiver_id, call_time)
VALUES
  (1, 2, TIMESTAMP '2022-07-04 10:13:49'),
  (1, 5, TIMESTAMP '2022-08-21 23:54:56'),
  (5, 1, TIMESTAMP '2022-05-13 17:24:06'),
  (5, 6, TIMESTAMP '2022-03-18 12:11:49');

SELECT * FROM `USE_YOUR_DATASET.phone_calls` 

"""
df_phone_calls = create_table(phone_calls)
df_phone_calls
phone_info = f"""
CREATE TABLE USE_YOUR_DATASET.phone_info (
  caller_id INT64,
  country_id STRING,
  network STRING,
  phone_number STRING
);

INSERT INTO USE_YOUR_DATASET.phone_info (caller_id, country_id, network, phone_number)
VALUES
  (1, 'US', 'Verizon', '+1-212-897-1964'),
  (2, 'US', 'Verizon', '+1-703-346-9529'),
  (3, 'US', 'Verizon', '+1-650-828-4774'),
  (4, 'US', 'Verizon', '+1-415-224-6663'),
  (5, 'IN', 'Vodafone', '+91 7503-907302'),
  (6, 'IN', 'Vodafone', '+91 2287-664895');

SELECT * FROM `USE_YOUR_DATASET.phone_info`

"""
df_phone_info = create_table(phone_info)
df_phone_info

,caller_id,country_id,network,phone_number
0,5,IN,Vodafone,+91 7503-907302
1,6,IN,Vodafone,+91 2287-664895
2,1,US,Verizon,+1-212-897-1964
3,3,US,Verizon,+1-650-828-4774
4,2,US,Verizon,+1-703-346-9529
5,4,US,Verizon,+1-415-224-6663


i added(JOIN) columns country_caller_id, country_receiver_id with phone_info.

In [13]:
international_calls = f"""
WITH phone_calls AS (
  SELECT * FROM `USE_YOUR_DATASET.phone_calls`
)
, phone_info AS (
  SELECT * FROM `USE_YOUR_DATASET.phone_info`
)
, phone_country AS (
  SELECT phone_calls.*
    , info_caller.country_id AS country_caller_id
    , info_receiver.country_id AS country_receiver_id
  FROM phone_calls
  LEFT JOIN phone_info AS info_caller ON phone_calls.caller_id = info_caller.caller_id
  LEFT JOIN phone_info AS info_receiver ON phone_calls.receiver_id = info_receiver.caller_id
)
, international_calls AS (
  SELECT *
    , if(country_caller_id = country_receiver_id, 1, 0) AS is_international_call 
  FROM phone_country
)
SELECT
  ROUND((SUM(is_international_call) / COUNT(*)) * 100) AS international_calls_pct
FROM international_calls

"""
df_international_calls = create_table(international_calls)
df_international_calls

,international_calls_pct
0,50.0


third sql task analog first/second

In [20]:
emails = f"""
CREATE TABLE USE_YOUR_DATASET.emails (
  email_id INT64,
  user_id INT64,
  signup_date TIMESTAMP
);

INSERT INTO USE_YOUR_DATASET.emails (email_id, user_id, signup_date)
VALUES
  (125, 7771, TIMESTAMP '2022-06-14 00:00:00'),
  (236, 6950, TIMESTAMP '2022-07-01 00:00:00'),
  (433, 1052, TIMESTAMP '2022-07-09 00:00:00');

SELECT * FROM `USE_YOUR_DATASET.emails`
"""
df_emails = create_table(emails)


texts = f"""
CREATE TABLE USE_YOUR_DATASET.texts (
  text_id INT64,
  email_id INT64,
  signup_action STRING
);

INSERT INTO USE_YOUR_DATASET.texts (text_id, email_id, signup_action)
VALUES
  (6878, 125, 'Confirmed'),
  (6920, 236, 'Not Confirmed'),
  (6994, 236, 'Confirmed');

SELECT * FROM `USE_YOUR_DATASET.texts`
"""
df_texts = create_table(texts)

df_emails
df_texts

,text_id,email_id,signup_action
0,6920,236,Not Confirmed
1,6994,236,Confirmed
2,6878,125,Confirmed


query 


In [21]:
email_activate= f"""
WITH emails AS (
  SELECT * FROM `USE_YOUR_DATASET.emails`
),
texts AS (
  SELECT * FROM `USE_YOUR_DATASET.texts`
),
last_texts AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY email_id ORDER BY text_id DESC) AS row_n
  FROM texts
),
email_activate AS (
  SELECT *
  FROM emails
  LEFT JOIN last_texts USING(email_id)
  WHERE last_texts.row_n = 1 OR last_texts.row_n IS NULL
),
confirmed_count AS (
  SELECT 
    SUM(IF(signup_action = "Confirmed", 1, 0)) AS confirmed
    , COUNT(*) AS total
  FROM email_activate
)
SELECT 
  ROUND(confirmed / total, 2) AS confirm_rate
FROM confirmed_count

"""
df_email_activate = create_table(email_activate)
df_email_activate

,confirm_rate
0,0.67


In [22]:
url_api = "USE_YOUR_API_URL"

In [33]:
# check API url and return json data
def get_data_from_api(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error: Unable API")
        return None

# convert to DataFrame format
def convert_to_dataframe(data):
    return pd.DataFrame(data)


# split to 500 rows in one file
def split_dataframe(df, chunk_size=500):
    num_chunks = (len(df) + chunk_size - 1) // chunk_size
    for i in range(num_chunks):
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(df))
        chunk = df.iloc[start:end]
        filename = f'data_{i + 1}.csv'
        chunk.to_csv(filename, index=False)
    
        print(f"Chunk {i + 1} saved to {filename}")


data = get_data_from_api(url_api)
if data:
    df = convert_to_dataframe(data)
    split_dataframe(df)
    print("Data download local")

Chunk 1 saved to data_1.csv
Chunk 2 saved to data_2.csv
Chunk 3 saved to data_3.csv
Chunk 4 saved to data_4.csv
Chunk 5 saved to data_5.csv
Chunk 6 saved to data_6.csv
Chunk 7 saved to data_7.csv
Chunk 8 saved to data_8.csv
Chunk 9 saved to data_9.csv
Chunk 10 saved to data_10.csv
Data download local
